In [ ]:
# pip install python-chess==0.31.3

# Creating the dataset

In [1]:
import chess

squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}


# example: h3 -> 17
def square_to_index(square):
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  # this is the 3d matrix
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # here we add the pieces's view on the matrix
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d

# TensorFlow!

In [7]:
"""
Initialize Tensorflow model
"""
import numpy
import tensorflow

model = tensorflow.keras.models.load_model('model_weight/global_model.h5')

# cen_model = tensorflow.keras.models.load_model("model_weight/centralized_weights")
# fed0_model = tensorflow.keras.models.load_model("model_weight/federated0_weights")
# fed1_model = tensorflow.keras.models.load_model("model_weight/federated1_weights")
# fed2_model = tensorflow.keras.models.load_model("model_weight/federated2_weights")


# Playing with the AI

In [11]:
# used for the minimax algorithm
def minimax_eval(board, model):
  board3d = split_dims(board)
  board3d = numpy.expand_dims(board3d, 0)
  return model.predict(board3d)[0][0]


def minimax(board, depth, alpha, beta, maximizing_player, model):
  if depth == 0 or board.is_game_over():
    return minimax_eval(board, model)
  
  if maximizing_player:
    max_eval = -numpy.inf
    for move in board.legal_moves:
      board.push(move)
      eval = minimax(board, depth - 1, alpha, beta, False, model)
      board.pop()
      max_eval = max(max_eval, eval)
      alpha = max(alpha, eval)
      if beta <= alpha:
        break
    return max_eval
  else:
    min_eval = numpy.inf
    for move in board.legal_moves:
      board.push(move)
      eval = minimax(board, depth - 1, alpha, beta, True, model)
      board.pop()
      min_eval = min(min_eval, eval)
      beta = min(beta, eval)
      if beta <= alpha:
        break
    return min_eval


# this is the actual function that gets the move from the neural network
def get_ai_move(board, depth, player, model):
  best_move = None
  max_eval = -numpy.inf
  min_eval = numpy.inf

  for move in board.legal_moves:
    board.push(move)
    eval = minimax(board, depth - 1, -numpy.inf, numpy.inf, False, model)
    board.pop()
    if player:
      if eval > max_eval:
        max_eval = eval
        best_move = move
    else:
      if eval < min_eval:
        min_eval = eval
        best_move = move
  return best_move

In [13]:
"""
AI vs AI
"""
board = chess.Board()

while True:
    move = get_ai_move(board, 1, True, model)
    board.push(move)
    print(move)
    print(f'\n{board}')
    if board.is_game_over():
        print('game_over')
        break
    move = get_ai_move(board, 1, False, model)
    board.push(move)
    print(move)
    print(f'\n{board}')
    if board.is_game_over():
        print('game_over')
        break


e2e4

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
e7e5

r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
d2d4

r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . P P . . .
. . . . . . . .
P P P . . P P P
R N B Q K B N R
f8b4

r n b q k . n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. b . P P . . .
. . . . . . . .
P P P . . P P P
R N B Q K B N R
c1d2

r n b q k . n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. b . P P . . .
. . . . . . . .
P P P B . P P P
R N . Q K B N R
e5d4

r n b q k . n r
p p p p . p p p
. . . . . . . .
. . . . . . . .
. b . p P . . .
. . . . . . . .
P P P B . P P P
R N . Q K B N R
d2b4

r n b q k . n r
p p p p . p p p
. . . . . . . .
. . . . . . . .
. B . p P . . .
. . . . . . . .
P P P . . P P P
R N . Q K B N R
d7d6

r n b q k . n r
p p p . . p p p
. . . p . . . .
. . . . 